# Trabajo práctico 9 - Calibración

**Alumnos:**

- Carol lugones Ignacio (100073)
- Torresetti Lisandro (99846)

## Objetivo

En base a las funciones de calibración implementadas en OpenCV realizar la calibración de sus celulares y devolver la matriz de parámetros intrinsecos.

In [ ]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
%matplotlib inline

In [ ]:
def plotter(image, title = '', imgSize = (18,9), grayScale = False, step = 100): #Funcion auxiliar para realizar los graficos
    plt.figure(figsize=imgSize)
    plt.title(title, fontsize = 16, fontweight = "bold")
    plt.imshow(image) if not grayScale else plt.imshow(image, cmap='gray', vmin=0, vmax=255)
    plt.yticks(np.arange(0, len(image), step))
    plt.xticks(np.arange(0, len(image[0]), step), rotation=90)
    plt.show() 

El patrón a analizar es el siguiente:

In [ ]:
pattern = cv.imread('ChessBoardPattern.png')
plotter(pattern, 'Chess Board Pattern')

Ahora el tamaño del tablero, y la lista de puntos a reconocer, tomando como origen (0, 0, 0) la esquina que se encuentra en las coordenadas (100, 100).

In [ ]:
chessBoardSize  = (7, 7)
objp = np.zeros((np.prod(chessBoardSize), 3),  dtype=np.float32)
objp[:, :2] = np.mgrid[0:chessBoardSize[0], 0:chessBoardSize[1]].T.reshape(-1, 2)

A continuación cargamos las imágenes tomadas al patrón de distintos ángulos.

In [ ]:
img_fnames = glob('./fotos/*')
imgsGray = []
imgsColor = []
for imgName in img_fnames:
    img = cv.imread(imgName)
    imgsColor.append(cv.cvtColor(img, cv.COLOR_BGR2RGB))
    imgsGray.append(cv.cvtColor(img, cv.COLOR_BGR2GRAY))
    

findChessboardCorners: a regular chessboard has 8 x 8 squares and 7 x 7 internal corners, that is, points where the black squares touch each other. The detected coordinates are approximate, and to determine their positions more accurately, the function calls cornerSubPix. You also may use the function cornerSubPix with different parameters if returned coordinates are not accurate enough.

Devuelve un booleano si encontro o no corners y despues los corners

In [ ]:
imgPoints = []
objPoints = []
def findCorners(imgsColor, imgsGray, maxCount = 25, epsilon = 0.001):
    criteria = (cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_MAX_ITER, maxCount, epsilon)
    cb_flags = cv.CALIB_CB_ADAPTIVE_THRESH  
    for imgColor, imgGray in zip(imgsColor, imgsGray):
        imgColor = imgColor.copy()
        ret, corners = cv.findChessboardCorners(imgGray, chessBoardSize, flags=cb_flags)
        if ret:
            print('Encontramos esquinas!')
            objPoints.append(objp)
            print('Buscando esquinas en resolución subpixel', end='... ')
            corners_subp = cv.cornerSubPix(imgGray, corners, (5, 5), (-1, -1), criteria)
            print('OK!')
            imgPoints.append(corners_subp)
            cv.drawChessboardCorners(imgColor, chessBoardSize, corners_subp, ret)
            plotter(imgColor)

In [ ]:
findCorners(imgsColor, imgsGray)

## Calibración

In [ ]:
h, w = imgsGray[0].shape
h, w

In [ ]:
import PIL.ExifTags
import PIL.Image

In [ ]:
ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objPoints, imgPoints, (w, h), None, None)

print('Camera Matrix = ')
print(mtx)
print('Distortion Coefficients = ')
print(dist)

In [ ]:
print('Leyendo datos del header EXIF ...')
exif_img = PIL.Image.open(img_fnames[2])

exif_data = {
 PIL.ExifTags.TAGS[k]: v
 for k, v in exif_img._getexif().items()
 if k in PIL.ExifTags.TAGS
}

print('Full exif dump:')
import pprint
pprint.pprint(exif_data)

#focal_length_exif = exif_data['FocalLength'] # tuple (p, q) -> p/q
#focal_length = focal_length_exif[0]/focal_length_exif[1]
#print('Distancia focal = ', focal_length, 'mm')